In [1]:
from qiskit import QuantumCircuit, QuantumRegister, AncillaRegister
from qiskit.circuit.library import MCPhaseGate, QFTGate 
from math import log2, ceil, pi

In [2]:
def draper_adder(a, num_of_qubits, num_of_controls, exclude_qft=False):
    ## First num_of_controls qubits are control qubits
    circuit = QuantumCircuit(num_of_qubits + num_of_controls, name=f"Add {a} ({num_of_controls}c+{num_of_qubits}q)")

    qft = None
    if not exclude_qft:
        qft = QFTGate(num_of_qubits)
        qft_indices = range(num_of_controls, num_of_qubits+num_of_controls)
        circuit.append(qft, qft_indices)

    for i in range(num_of_qubits):
        current_phase = 2 * pi * a / (2**(num_of_qubits - i))
        if num_of_controls > 0:
            circuit.mcp(current_phase, list(range(0, num_of_controls)), i+num_of_controls)
        else:
            circuit.p(current_phase, i)

    if not exclude_qft:
        circuit.append(qft.inverse(), qft_indices)

    return circuit.to_gate()


In [3]:
def modular_adder(a, N, num_of_qubits):
    num_of_controls = 2
    circuit = QuantumCircuit(num_of_qubits 
                             + 1 #ancilla qubit
                             + 1 #overflow qubit
                             + num_of_controls, name=f"Modular add {a} mod {N} ({num_of_qubits}q)")

    a_adder = draper_adder(a, num_of_qubits + 1, num_of_controls, exclude_qft=True)
    circuit.append(a_adder, range(0, num_of_controls + num_of_qubits + 1))
    minus_N_adder = draper_adder(-N, num_of_qubits + 1, 0, exclude_qft=True)
    circuit.append(minus_N_adder, range(num_of_controls, num_of_controls + num_of_qubits + 1))
    
    # Perform inverse Quantum Fourier Transform in order to read overflow bit
    qft = QFTGate(num_of_qubits + 1)
    qft_indices = range(num_of_controls, num_of_qubits+num_of_controls+1)
    circuit.append(qft.inverse(), qft_indices)

    overflow_bit_index = num_of_controls + num_of_qubits
    ancilla_bit_index = overflow_bit_index + 1
    # Use the overflow bit to check if we need to add N-a (the inverse of adding a-N)
    circuit.cx(overflow_bit_index, ancilla_bit_index)

    circuit.append(qft, qft_indices)
    # add N back if we weren't greater than N
    N_adder_with_control = draper_adder(N, num_of_qubits + 1, num_of_controls=1, exclude_qft=True)

    # restore ancilla - this is no Joke
    circuit.append(N_adder_with_control, [ancilla_bit_index] + list(qft_indices))
    circuit.append(a_adder.inverse(), range(0, num_of_controls + num_of_qubits + 1))
    circuit.append(qft.inverse(), qft_indices)
    circuit.x(overflow_bit_index)
    circuit.cx(overflow_bit_index, ancilla_bit_index)
    circuit.x(overflow_bit_index)
    circuit.append(qft, qft_indices)
    circuit.append(a_adder, range(0, num_of_controls + num_of_qubits + 1))

    return circuit.to_gate()

In [4]:
def modular_multiplier(a, N, num_of_qubits):
    circuit = QuantumCircuit(1 #control                            
                             + num_of_qubits #x input
                             + num_of_qubits #b input
                             + 1 # overflow
                             + 1, # ancilla 
                             name=f"Modular multiply {a} mod {N} ({num_of_qubits}q)")
    
    control_qubit_index = 0
    overflow_qubit_index = 2*num_of_qubits + 1
    ancilla_qubit_index = 2*num_of_qubits + 2
    b_indices = list(range(num_of_qubits + control_qubit_index + 1, 2*num_of_qubits+control_qubit_index+1)) + [overflow_qubit_index]

    qft = QFTGate(num_of_qubits + 1)
    circuit.append(qft, b_indices)

    for i in range(0, num_of_qubits):
        current_modular_adder = modular_adder(2**i * a, N, num_of_qubits)
        adder_qubits = [control_qubit_index, control_qubit_index + i + 1] + b_indices + [ancilla_qubit_index]
        circuit.append(current_modular_adder, adder_qubits)

    circuit.append(qft.inverse(), b_indices)
    return circuit.to_gate()

In [5]:
from qiskit.quantum_info import Statevector

myCircuit = QuantumCircuit(11)
myCircuit.x(0)
myCircuit.x(1)
myCircuit.x(2)
myMultiplier = modular_multiplier(3,5,4)
myCircuit.append(myMultiplier, range(0, 11))
myCircuit.draw()
 
state = Statevector.from_instruction(myCircuit)
state.probabilities_dict()

{np.str_('00000000111'): np.float64(1.9561862320090797e-30),
 np.str_('00000100111'): np.float64(3.6540533339490806e-31),
 np.str_('00001000111'): np.float64(1.560723431373235e-29),
 np.str_('00001100111'): np.float64(2.809045807276015e-29),
 np.str_('00010000111'): np.float64(0.9999999999999882),
 np.str_('00010100111'): np.float64(7.395854244375752e-30),
 np.str_('00011000111'): np.float64(9.875465259548975e-30),
 np.str_('00011100111'): np.float64(1.3232571423388326e-30),
 np.str_('00100000111'): np.float64(1.531067103840799e-30),
 np.str_('00100100111'): np.float64(5.601608582944111e-31),
 np.str_('00101000111'): np.float64(1.1459970192059912e-30),
 np.str_('00101100111'): np.float64(6.155675197234958e-32),
 np.str_('00110000111'): np.float64(2.821846013134326e-31),
 np.str_('00110100111'): np.float64(3.3424577606429865e-30),
 np.str_('00111000111'): np.float64(1.724683501479532e-30),
 np.str_('00111100111'): np.float64(8.992749279703538e-32),
 np.str_('01000000111'): np.float64(3.